In [25]:
# 包含车速信息，代码运行很慢，因为有车速信息，所以必须要逐帧处理

import cv2
import numpy as np
import time
import pandas as pd
import os
import torch
from tqdm import tqdm



In [26]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.to('cpu')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [27]:
# 定义检测的类（只检测车辆相关的类别）
classes_to_detect = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

In [28]:
from tools.video import get_video_paths

video_paths = get_video_paths("./dataset/train")

print(video_paths)


['./dataset/train/32.31.250.105/105_combined.mp4', './dataset/train/32.31.250.105/20240501_20240501115227_20240501130415_115227.mp4', './dataset/train/32.31.250.105/20240501_20240501130415_20240501141554_130415.mp4', './dataset/train/32.31.250.105/20240501_20240501141554_20240501152820_141555.mp4', './dataset/train/32.31.250.103/20240501_20240501140806_20240501152004_140807.mp4', './dataset/train/32.31.250.103/103_combined.mp4', './dataset/train/32.31.250.103/20240501_20240501125647_20240501140806_125649.mp4', './dataset/train/32.31.250.108/108_combined.mp4', './dataset/train/32.31.250.108/20240501_20240501113543_20240501135236_113542.mp4', './dataset/train/32.31.250.108/20240501_20240501135236_20240501160912_135235.mp4', './dataset/train/32.31.250.107/20240501_20240501135755_20240501161432_135755.mp4', './dataset/train/32.31.250.107/20240501_20240501114103_20240501135755_114103.mp4', './dataset/train/32.31.250.107/107_combined.mp4']


In [29]:
for path in tqdm(video_paths):
    file_folder = path.split('/')[-2]
    full_file_folder = str('/').join(path.split('/')[:-1])
    # 打开视频文件
    filename = os.path.basename(path)
    video_name = os.path.splitext(filename)[0]
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频帧率

    # 获取视频的帧数
    all_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"视频总帧数: {all_frame_count}")

    # 定义用于计算流量的线的位置
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    line_position = frame_height // 2  # 根据需要调整位置

    # 初始化变量
    frame_count = 0
    output_data = []

    # 定义处理间隔（每25帧处理一次）
    process_frame_interval = 25  # 每25帧处理一次

    # 定义输出间隔（以处理的帧数为单位）
    output_frame_interval = 1  # 每处理25帧输出一次

    processed_frame_count = 0  # 处理过的帧数计数器

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # 仅每隔指定的帧数处理一次
        if frame_count % process_frame_interval != 0:
            continue

        processed_frame_count += 1  # 增加处理过的帧数计数器

        # 使用YOLOv8模型进行推理
        results = model.predict(frame)  # YOLOv8的推理方法
        detections = results[0].boxes  # 获取第一个结果的检测框

        boxes = []
        confidences = []
        class_ids = []

        for box in detections:
            confidence = box.conf.item()  # 获取置信度
            class_id = int(box.cls.item())  # 获取类别ID
            if confidence > 0.3:  # 使用置信度阈值
                class_name = model.names[class_id]  # 获取类别名称
                if class_name in classes_to_detect:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # 获取边界框坐标
                    boxes.append([x1, y1, x2 - x1, y2 - y1])
                    confidences.append(confidence)
                    class_ids.append(class_id)

        # 计算密度：当前帧中的车辆数量
        density = len(boxes)

        # 计算流量：判断位于指定线位置的车辆数量
        flow = 0
        for box in boxes:
            x1, y1, w, h = box
            y2 = y1 + h
            # 检查车辆是否与线相交
            if y1 <= line_position <= y2:
                flow += 1

        # 每隔一定的处理帧数输出一次数据
        if processed_frame_count % output_frame_interval == 0:
            output = {
                'Frame': frame_count,
                'Flow': flow,
                'Density': density
            }
            print(f"视频总帧数: {all_frame_count}")
            print(output)
            output_data.append(output)

    cap.release()

    # 保存结果到文件
    df = pd.DataFrame(output_data)
    saved_folder = f"./res/without-speed/{file_folder}"
    if not os.path.exists(saved_folder):
        os.makedirs(saved_folder, exist_ok=True)
    df.to_csv(f'./res/without-speed/{file_folder}/{video_name}.csv', index=False)


  0%|          | 0/13 [00:00<?, ?it/s]

视频总帧数: 32385
0: 384x640 4 cars, 2 buss, 1 truck, 1 traffic light, 66.0ms
Speed: 2.2ms preprocess, 66.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 25, 'Flow': 1, 'Density': 7}
0: 384x640 11 cars, 1 truck, 1 traffic light, 51.1ms
Speed: 1.4ms preprocess, 51.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 50, 'Flow': 1, 'Density': 12}
0: 384x640 6 cars, 2 trucks, 1 traffic light, 58.6ms
Speed: 1.9ms preprocess, 58.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 75, 'Flow': 2, 'Density': 8}
0: 384x640 7 cars, 2 trucks, 52.8ms
Speed: 1.4ms preprocess, 52.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 100, 'Flow': 2, 'Density': 9}
0: 384x640 5 cars, 2 trucks, 1 traffic light, 51.5ms
Speed: 1.6ms preprocess, 51.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 125, 'Flow': 1

  0%|          | 0/13 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [30]:
import time
import pandas as pd
import os
import cv2
from tqdm import tqdm
import csv

# 初始化变量
all_output_data = []  # 存储所有视频的运行数据

for path in tqdm(video_paths):
    file_folder = path.split('/')[-2]
    full_file_folder = str('/').join(path.split('/')[:-1])
    
    # 打开视频文件
    filename = os.path.basename(path)
    video_name = os.path.splitext(filename)[0]
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频帧率

    # 获取视频的帧数
    all_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"视频总帧数: {all_frame_count}")

    # 定义用于计算流量的线的位置
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    line_position = frame_height // 2  # 根据需要调整位置

    # 初始化变量
    frame_count = 0
    output_data = []

    # 定义处理间隔（每25帧处理一次）
    process_frame_interval = 25  # 每25帧处理一次

    # 定义输出间隔（以处理的帧数为单位）
    output_frame_interval = 1  # 每处理25帧输出一次

    processed_frame_count = 0  # 处理过的帧数计数器

    # 记录开始时间
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # 仅每隔指定的帧数处理一次
        if frame_count % process_frame_interval != 0:
            continue

        processed_frame_count += 1  # 增加处理过的帧数计数器

        # 使用YOLOv8模型进行推理
        results = model.predict(frame)
        detections = results[0].boxes

        boxes = []
        confidences = []
        class_ids = []

        for box in detections:
            confidence = box.conf.item()
            class_id = int(box.cls.item())
            if confidence > 0.3:  # 使用置信度阈值
                class_name = model.names[class_id]
                if class_name in classes_to_detect:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # 获取边界框坐标
                    boxes.append([x1, y1, x2 - x1, y2 - y1])
                    confidences.append(confidence)
                    class_ids.append(class_id)

        # 计算密度：当前帧中的车辆数量
        density = len(boxes)

        # 计算流量：判断位于指定线位置的车辆数量
        flow = 0
        for box in boxes:
            x1, y1, w, h = box
            y2 = y1 + h
            if y1 <= line_position <= y2:
                flow += 1

        # 每隔一定的处理帧数输出一次数据
        if processed_frame_count % output_frame_interval == 0:
            output = {
                'Frame': frame_count,
                'Flow': flow,
                'Density': density
            }
            print(f"视频总帧数: {all_frame_count}")
            print(output)
            output_data.append(output)

    # 记录结束时间
    end_time = time.time()

    # 计算总处理时间
    total_time = end_time - start_time

    # 计算处理帧率（每秒处理帧数）
    process_fps = processed_frame_count / total_time

    # 保存每个视频的统计数据
    video_output = {
        'Video Name': video_name,
        'Total Frames': all_frame_count,
        'Processed Frames': processed_frame_count,
        'Total Time (s)': total_time,
        'Process FPS': process_fps
    }
    all_output_data.append(video_output)

    # 将当前视频的输出保存到CSV文件
    # df = pd.DataFrame(output_data)
    # saved_folder = f"./res/without-speed/{file_folder}"
    # if not os.path.exists(saved_folder):
    #     os.makedirs(saved_folder, exist_ok=True)
    # df.to_csv(f'./res/without-speed/{file_folder}/{video_name}.csv', index=False)

    cap.release()

# 保存所有视频的运行数据到CSV文件
output_csv_path = "yolov8_runtime_stats.csv"
with open(output_csv_path, mode='w', newline='') as csv_file:
    fieldnames = ['Video Name', 'Total Frames', 'Processed Frames', 'Total Time (s)', 'Process FPS']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for data in all_output_data:
        writer.writerow(data)

print(f"所有视频处理数据已保存到 {output_csv_path}")


  0%|          | 0/13 [00:00<?, ?it/s]

视频总帧数: 32385
0: 384x640 4 cars, 2 buss, 1 truck, 1 traffic light, 66.7ms
Speed: 1.2ms preprocess, 66.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 25, 'Flow': 1, 'Density': 7}
0: 384x640 11 cars, 1 truck, 1 traffic light, 63.2ms
Speed: 1.2ms preprocess, 63.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 50, 'Flow': 1, 'Density': 12}
0: 384x640 6 cars, 2 trucks, 1 traffic light, 50.4ms
Speed: 1.2ms preprocess, 50.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 75, 'Flow': 2, 'Density': 8}
0: 384x640 7 cars, 2 trucks, 55.0ms
Speed: 1.6ms preprocess, 55.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 100, 'Flow': 2, 'Density': 9}
0: 384x640 5 cars, 2 trucks, 1 traffic light, 53.4ms
Speed: 1.4ms preprocess, 53.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 32385
{'Frame': 125, 'Flow': 1

  8%|▊         | 1/13 [01:57<23:28, 117.33s/it]

视频总帧数: 32385
{'Frame': 32375, 'Flow': 0, 'Density': 3}
视频总帧数: 10770
0: 384x640 4 cars, 2 buss, 1 truck, 1 traffic light, 52.7ms
Speed: 2.3ms preprocess, 52.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 10770
{'Frame': 25, 'Flow': 1, 'Density': 7}
0: 384x640 11 cars, 1 truck, 1 traffic light, 52.0ms
Speed: 1.8ms preprocess, 52.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 10770
{'Frame': 50, 'Flow': 1, 'Density': 12}
0: 384x640 6 cars, 2 trucks, 1 traffic light, 52.2ms
Speed: 1.6ms preprocess, 52.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 10770
{'Frame': 75, 'Flow': 2, 'Density': 8}
0: 384x640 7 cars, 2 trucks, 55.0ms
Speed: 1.6ms preprocess, 55.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 10770
{'Frame': 100, 'Flow': 2, 'Density': 9}
0: 384x640 5 cars, 2 trucks, 1 traffic light, 50.3ms
Speed: 1.6ms preprocess, 50.3ms inference, 0.4ms postprocess per image at shape

 15%|█▌        | 2/13 [02:36<13:02, 71.12s/it] 

...

视频总帧数: 27325
{'Frame': 27325, 'Flow': 0, 'Density': 9}
视频总帧数: 27322
0: 384x640 6 cars, 70.4ms
Speed: 2.3ms preprocess, 70.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 27322
{'Frame': 25, 'Flow': 1, 'Density': 6}
0: 384x640 10 cars, 61.0ms
Speed: 1.9ms preprocess, 61.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 27322
{'Frame': 50, 'Flow': 1, 'Density': 10}
0: 384x640 9 cars, 59.1ms
Speed: 1.4ms preprocess, 59.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 27322
{'Frame': 75, 'Flow': 1, 'Density': 9}
0: 384x640 12 cars, 58.5ms
Speed: 2.3ms preprocess, 58.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 27322
{'Frame': 100, 'Flow': 1, 'Density': 12}
0: 384x640 8 cars, 59.2ms
Speed: 1.4ms preprocess, 59.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 27322
{'Frame': 125, 'Flow': 1, 'Density': 8}
0: 384x640 11 cars, 1 truck, 65.2ms
Speed: 1.9ms pr

 92%|█████████▏| 12/13 [17:11<01:52, 112.08s/it]

视频总帧数: 54647
0: 384x640 6 cars, 92.0ms
Speed: 2.4ms preprocess, 92.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 54647
{'Frame': 25, 'Flow': 1, 'Density': 6}
0: 384x640 10 cars, 76.6ms
Speed: 3.2ms preprocess, 76.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 54647
{'Frame': 50, 'Flow': 1, 'Density': 10}
0: 384x640 9 cars, 96.9ms
Speed: 3.1ms preprocess, 96.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 54647
{'Frame': 75, 'Flow': 1, 'Density': 9}
0: 384x640 12 cars, 68.0ms
Speed: 2.4ms preprocess, 68.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 54647
{'Frame': 100, 'Flow': 1, 'Density': 12}
0: 384x640 8 cars, 65.6ms
Speed: 1.4ms preprocess, 65.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
视频总帧数: 54647
{'Frame': 125, 'Flow': 1, 'Density': 8}
0: 384x640 11 cars, 1 truck, 64.9ms
Speed: 1.9ms preprocess, 64.9ms inference, 0.4ms postprocess per image

100%|██████████| 13/13 [21:01<00:00, 97.00s/it] 

所有视频处理数据已保存到 yolov8_runtime_stats.csv
